# Spark DataFrame Extra Task (готовий розвʼязок)

Вимоги ТЗ:
- **тільки DataFrame API** (без SQL-рядків)
- локальний Spark `master('local[3]')`
- результат кожної задачі відтворюється в ноутбуці через `.show()`

Дані очікуються в папці `data/` (можуть бути й у підпапках — ноутбук знайде файли).


In [ ]:
from pyspark.sql import SparkSession, functions as F
from pathlib import Path


In [ ]:
# Spark session (LOCAL)
spark = (
    SparkSession.builder
    .master("local[3]")
    .appName("spark-hw")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")
print("Spark:", spark.version)


In [ ]:
# Robust repo/data detection
cwd = Path.cwd().resolve()
repo_root = None
for p in [cwd, *cwd.parents]:
    if (p / "data").exists():
        repo_root = p
        break
if repo_root is None:
    repo_root = cwd

REPO_ROOT = repo_root
DATA_DIR = (REPO_ROOT / "data").resolve()

print("CWD:", cwd)
print("Repo:", REPO_ROOT)
print("Data dir:", DATA_DIR, "exists:", DATA_DIR.exists())


In [ ]:
def locate_file(name: str, base: Path = None) -> Path | None:
    """Find file under base folder (recursive, case-insensitive)."""
    if base is None:
        base = DATA_DIR
    base = Path(base)
    if not base.exists():
        return None

    p = base / name
    if p.exists():
        return p

    lower = name.lower()
    for x in base.iterdir():
        if x.name.lower() == lower:
            return x

    hits = list(base.rglob(name))
    if hits:
        return hits[0]

    for x in base.rglob("*"):
        if x.is_file() and x.name.lower() == lower:
            return x
    return None

def read_csv(name: str):
    path = locate_file(name)
    if path is None:
        raise FileNotFoundError(f"{name} not found under {DATA_DIR}")
    return spark.read.csv(str(path), header=True, inferSchema=True)


## Load tables (CSV)


In [ ]:
actor_df = read_csv('actor.csv')
address_df = read_csv('address.csv')
category_df = read_csv('category.csv')
city_df = read_csv('city.csv')
country_df = read_csv('country.csv')
customer_df = read_csv('customer.csv')
film_df = read_csv('film.csv')
film_actor_df = read_csv('film_actor.csv')
film_category_df = read_csv('film_category.csv')
inventory_df = read_csv('inventory.csv')
language_df = read_csv('language.csv')
payment_df = read_csv('payment.csv')
rental_df = read_csv('rental.csv')
staff_df = read_csv('staff.csv')
store_df = read_csv('store.csv')

print('Tables loaded')


# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`).
- Використовувати SQL-рядки заборонено — **тільки DataFrame API**.


## 1. Вивести кількість фільмів в кожній категорії. Результат відсортувати за спаданням.


In [ ]:
task1 = (
    film_category_df
    .join(category_df, on='category_id', how='inner')
    .groupBy('name')
    .agg(F.countDistinct('film_id').alias('films_cnt'))
    .orderBy(F.desc('films_cnt'))
)
task1.show(truncate=False)


## 2. Вивести 10 акторів, чиї фільми брали на прокат найбільше. Результат відсортувати за спаданням.


In [ ]:
task2 = (
    rental_df
    .join(inventory_df.select('inventory_id', 'film_id'), on='inventory_id', how='inner')
    .join(film_actor_df, on='film_id', how='inner')
    .join(actor_df, on='actor_id', how='inner')
    .groupBy('actor_id', 'first_name', 'last_name')
    .agg(F.count('*').alias('rentals_cnt'))
    .orderBy(F.desc('rentals_cnt'))
    .limit(10)
)
task2.show(truncate=False)


## 3. Вивести категорію фільмів, на яку було витрачено найбільше грошей в прокаті


In [ ]:
task3 = (
    payment_df
    .join(rental_df.select('rental_id', 'inventory_id'), on='rental_id', how='inner')
    .join(inventory_df.select('inventory_id', 'film_id'), on='inventory_id', how='inner')
    .join(film_category_df, on='film_id', how='inner')
    .join(category_df, on='category_id', how='inner')
    .groupBy('name')
    .agg(F.sum('amount').alias('total_amount'))
    .orderBy(F.desc('total_amount'))
    .limit(1)
)
task3.show(truncate=False)


## 4. Вивести назви фільмів, яких не має в inventory.


In [ ]:
task4 = (
    film_df
    .select('film_id', 'title')
    .join(inventory_df.select('film_id').distinct(), on='film_id', how='left_anti')
    .orderBy('title')
)
task4.show(truncate=False)


## 5. Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”


In [ ]:
children_cat = category_df.filter(F.col('name') == F.lit('Children')).select('category_id')

task5 = (
    film_category_df
    .join(children_cat, on='category_id', how='inner')
    .join(film_actor_df, on='film_id', how='inner')
    .join(actor_df, on='actor_id', how='inner')
    .groupBy('actor_id', 'first_name', 'last_name')
    .agg(F.countDistinct('film_id').alias('films_in_children'))
    .orderBy(F.desc('films_in_children'))
    .limit(3)
)
task5.show(truncate=False)


## Stop Spark session


In [ ]:
spark.stop()
print('Spark stopped')
